# Exercise: Create and upload an OLAP Cube with mstrio

## Introduction & Library Imports

You are a developer at a Cryptocurrency firm, Blockchain Unity. As part of the platform, the price data of various cryptocurrencies are automatically scraped with a web crawler and saved in a comma delimited format. This data then needs to be uploaded to MicroStrategy and refreshed every hour. To improve efficiency, you are tasked with adding this data to an OLAP cube and uploading it to the MicroStrategy metadata. Instead of manually uploading every single data file, you use mstrio to automatically create and upload the cubes.

## Declare environment credential variables

Begin by declaring some variables that holds the environment details. Since we are using Standard Authentication using the mstr username and password from the Welcome to MicroStrategy Email, we can store this information alongside the environment ID, all required to establish a connection with the Rest API through mstrio.

In [1]:
# Input credentials

env=input("ENV ID: ")
username=input("Username: ")
pw = input("Password: ")

ENV ID: 309598
Username: mstr
Password: ISvFDIk9mGqn


## Establish an environment connection

With the variables stored, we need to establish a connection with the MicroStrategy environment. This is done by creating a Connection object from the mstrio.connection class. The connection object takes in the base REST API url, the username, and password. The REST API url is generated using the environment number we saved previously.

In [2]:
# Connect to your MSTR environment

from mstrio.connection import Connection
from mstrio.server import Project, compare_project_settings, Environment

base_url = "https://env-" + env + ".customer.cloud.microstrategy.com/MicroStrategyLibrary/api"
username = username
password = pw
conn = Connection(base_url, username, password, project_name="MicroStrategy Tutorial", login_mode=1)
env = Environment(connection=conn)

Connection to MicroStrategy Intelligence Server has been established.


## Import additional mstrio modules

Now that a connection has been established, additional modules are required to get information on the cubes within any MicroStrategy project. In this exercise, all new cubes are saved in the MicroStrategy Tutorial project.

In [3]:
# Import additional OLAP Cube, object managment, utility modules

from mstrio.object_management import (Folder, list_folders, get_predefined_folder_contents,
                                    get_my_personal_objects_contents, PredefinedFolders, Object,
                                    list_objects, SearchObject, get_search_suggestions,
                                    quick_search, quick_search_from_object, full_search,
                                    start_full_search, get_search_results, SearchPattern,
                                    SearchResultsFormat)
from mstrio.project_objects.dossier import Dossier
from mstrio.utils.entity import Rights
from mstrio.types import ObjectSubTypes, ObjectTypes
import json
from mstrio.project_objects.datasets.olap_cube import OlapCube, list_olap_cubes

print("Imported required modules")

Imported required modules


## List folders of the MicroStrategy Tutorial Project

To get the folder IDs of the MicroStrategy Tutorial Project, we must first get the project that's loaded.

In [4]:
# Establish MicroStrategy Tutorial project and list loaded projects

loaded_projects = env.list_loaded_projects()
for project in loaded_projects:
    print(project)

tutorial_id = input("Copy and Paste the Microstrategy Tutorial Project ID: ")

Project object named: 'MicroStrategy Tutorial' with ID: 'B7CA92F04B9FAE8D941C3E9B7E0CD754'
Project object named: 'Consolidated Education Project' with ID: 'CE52831411E696C8BD2F0080EFD5AF44'
Project object named: 'Hierarchies Project' with ID: 'B3FEE61A11E696C8BD0F0080EFC58F44'
Project object named: 'Human Resources Analysis Module' with ID: '4BAE16A340B995CAD24193AA3AC15D29'
Project object named: 'Relationships Project' with ID: '4C09350211E69712BAEE0080EFB56D41'
Project object named: 'Platform Analytics' with ID: '0DDDDEC8C94B320B4E93498C1EE98D18'
Copy and Paste the Microstrategy Tutorial Project ID: B7CA92F04B9FAE8D941C3E9B7E0CD754


Now that we have the ID of the MicroStrategy Tutorial Project, we can list the folders of the project using the project ID.

In [5]:
# List folders from a particular project
list_folders(conn, project_id=tutorial_id)

[Folder(connection, id='4A73885B8ACA4A65BB2AD3AA3E1DE576', name='Object Templates'),
 Folder(connection, id='84DAAC900B864173907F44B75C68FD96', name='Profiles'),
 Folder(connection, id='42EEDD41A6954F7485453C170AA3F8BE', name='Project Builder'),
 Folder(connection, id='02C37D85EE25483AA5708E2BFE858B92', name='Project Objects'),
 Folder(connection, id='98FE182C2A10427EACE0CD30B6768258', name='Public Objects'),
 Folder(connection, id='95C3B713318B43D490EE789BE27D298C', name='Schema Objects'),
 Folder(connection, id='09AA41BA555349E8B6E5C750073DBDE9', name='System Objects'),
 Folder(connection, id='14B3EE8211D540A5C0005E916B98494F', name='Version Information'),
 Folder(connection, id='C849EA5311D49BBCC0003486395B684F', name='Version Update History')]

Now that we have enough information on where to put our cube, let's build a quick example cube with one attribute and one metric. Note that these IDs correspond to default ones in the tutorial, Customer and Prices. If we wanted actual cryptocurrency values, we would need to create separate objects with these IDs first in order to build our cube.

In [6]:
folder = '98FE182C2A10427EACE0CD30B6768258'

# Create new OLAP Cube using a single attribute and metric
attributes = [{
    'id': '8D679D3C11D3E4981000E787EC6DE8A4',
    'name': 'Ticker',
    'type': 'attribute',
    'forms': [{
        'id': '8D67A35B11D3E4981000E787EC6DE8A4',
        'name': 'Ticker DESC 1',
        'type': 'form'
    }]
}]
metrics = [{'id': '5E2B495C11D66F4BC0006CBEB630224F', 'name': 'Price', 'type': 'metric'}]
new_olap_cube = OlapCube.create(conn, name="CryptoCurrency Ticker Prices", description="A price list of some cryptocurrencies",
                                folder_id=folder, attributes=attributes, metrics=metrics)

## Publish, refresh, and show status of the cube

Now that we have an example cube, we can publish it. This enables the cube to be used as a dataset.

In [7]:
# Publish Cube
new_olap_cube.publish()

Request for publishing cube 'CryptoCurrency Ticker Prices' was sent.


{'id': '131:RU5WLTMwOTU5OExBSU9VU0Ux', 'jobId': 131}

Lastly, we can refresh and show the status of our cube.

In [10]:
# Refresh and show status of an OLAP Cube
new_olap_cube.refresh_status()
new_olap_cube.show_status()


Ready, Loaded, Persisted, Active, Reserved


['Ready', 'Loaded', 'Persisted', 'Active', 'Reserved']